<font size="5">Import modules</font>

In [ ]:
import pandas as pd
import numpy as np
import time

<font size="5">Import data from CSV file</font>

In [2]:
df = pd.read_csv("final_edited.csv", error_bad_lines=False, sep=';')

In [40]:
df = pd.read_csv("all_linkedin_companies_export.csv", low_memory=False)

<font size="5">Import data for our comparison dictionary from CSV file</font>

In [2]:
df_voc = pd.read_csv("Country_voc.csv")

<font size="5">Print out file columns and check if column contains non-unique values</font>

In [ ]:
df_voc["sovereignty"].value_counts()
df_voc["primary_language"].value_counts()

<font size="5">Fill empty cells witn N/a and create dictionary for each country name variant. Dictionary keys set to 'country_standard' column</font>

In [4]:
df_voc = df_voc.fillna("N/a")

In [5]:
country_dict = df_voc.set_index('country_standard').to_dict()["iso_country_name_orignial"]
country_dict1 = df_voc.set_index('country_standard').to_dict()["iso_country_no_x"]
country_dict2 = df_voc.set_index('country_standard').to_dict()["iso_country_cleaned"]
country_dict3 = df_voc.set_index('country_standard').to_dict()["official_state_name"]
country_dict4 = df_voc.set_index('country_standard').to_dict()["shoprank_country"]
country_dict5 = df_voc.set_index('country_standard').to_dict()["ns8_country"]
country_dict6 = df_voc.set_index('country_standard').to_dict()["similarweb_export_country"]
country_dict7 = df_voc.set_index('country_standard').to_dict()["linkedin_country"]

<font size="5">Combine all dictionaries into one, with the same dictionary key</font>

In [14]:
final_dict = {}
list_dict = [country_dict, country_dict1, country_dict2, country_dict3, country_dict4, country_dict5, country_dict6, country_dict7]

for d in list_dict:
    for key in d:
        if key not in final_dict:
            final_dict[key] = []
        final_dict[key].append(d[key])

<font size="5">Delete identical values from dictionary values, to make future iterations faster</font>

In [ ]:
final_dict = {k:list(set(j)) for k,j in final_dict.items()}
print(final_dict)

<font size="5">Create function that will return dictionary key (our needed value for replacement) if one of country name variant will be found in files</font>

In [16]:
def get_key(val):
    for key, values in final_dict.items():
        for item in values:
            if val == item:
                return key
    return "No_such_country"

<font size="5">Quick check if the functions works well</font>

In [17]:
get_key('Afghanistan')

'Afghanistan'

<font size="5">Create list from column of file that we want to make replacement in</font>

In [18]:
values_list = df['cond_country'].tolist()

<font size="5">Iterate throught each row value from selected column and return correct name of country, this names appended to a new list - test_list</font>

In [20]:
n = 0
test_list = []
for i in values_list:
    a = get_key(i)
    test_list.append(a)
    n += 1

<font size="5">Create a new column from test_list data</font>

In [21]:
df["new_values"] = test_list

<font size="5">Make replacement for target column and column of our test_list values - "new_values" column. It could be done by for loop and apply() function, but the second variant turns out 50 times faster</font>

In [ ]:
start = time.time()
for i in range(len(df)):
    if df.new_values[i] != 'No_such_country':
        df.server_country[i] = df.new_values[i]
end = time.time()-start
print("Execution time", end)

In [22]:
start = time.time()
def label_row (row):
    if row["new_values"] != "No_such_country":
        return row["new_values"]
    else:
        return row["cond_country"]

df["cond_country"] = df.apply(lambda row: label_row(row), axis=1)
end = time.time()-start
print("Execution time", end)

Execution time 255.44512033462524


<font size="5">Quick check if replacement was done correctly. All values should be "Match"</font>

In [ ]:
def label_row (row):
    if row["cond_country"] == row["new_values"]:
        return "Match"
    else:
        return "NO"

df["Match_check"] = df.apply(lambda row: label_row(row), axis=1)
df["Match_check"].value_counts()

<font size="5">Drop technically created columns before saving the file</font>

In [32]:
df.drop(['new_values'], axis=1, inplace=True)

In [4]:
df.drop(["Match_check"], axis=1, inplace=True)

<font size="5">Saving the file into CSV format</font>

In [25]:
df.to_csv("all_linkedin_companies_export_done.csv", index=False)